# Convert source ids from one region to another

In [1]:
from astropy import units as uu
#First region, usually the old one
region1_name = "ALL_10-800_LOOSE"  #"ALL_10-800" #"CAINGSGII_10-800"  #"TDS" #"CAINGSGII_10-800"  #"TDS" # "MDIS_10-800" # "TDS" # _ELAISN1
region1_fname = "./vasca_pipeline/"+region1_name+"/region_"+region1_name+"_cat.fits"

#Source IDs to be converted
srcs_ids1 = [4993 ,13580 ,14504 ,30475 ,50388 ,52994 ,79477 ,120265 ,138783 ,139480 ,162697 ,204419 ,221349 ,228958 ,236839 ,253214 ,253670
            ,256433 ,261213 ,331466 ,340464 ,381336 ,390864 ,489430 ,498065 ,525310 ,532554 ,533185 ,540235 ,583838 ,639720 ,645215 ,645730
            ,659787 ,666224 ,673175 ,680119 ,691408 ,714140 ,764827 ,1037284 ,1069077 ,1070938 ,1805505 ,1839206 ,2527679 ,3074917 ,] # WD ALL_10-800-LOOSE

#Second region, usually the new one
region2_name = "ALL_10-800"
region2_fname = "./vasca_pipeline/"+region2_name+"/region_"+region2_name+"_cat.fits"

#Maximum distance to be considered the same source
dist_max = 1*uu.arcsec

In [2]:
%matplotlib widget
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

from astropy.coordinates import SkyCoord
from astropy.table import unique, Table, hstack, join, setdiff
from astroquery.simbad import Simbad
from astropy.visualization import quantity_support
from astropy.modeling.models import BlackBody
from astropy import constants as cc
from matplotlib.ticker import ScalarFormatter
from astroquery.sdss import SDSS

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis
from vasca.tables import TableCollection, dd_vasca_columns
from vasca.utils import otype2ogroup,dd_ogrp2otypes, dd_ogrp2col, dd_filter2wavelength, mag2flux, flux2mag, query_vizier_sed, add_ogrp

from loguru import logger
#logger.enable("vasca")

In [3]:
rg1 = Region()
rg1.load_from_fits(region1_fname)
rg2 = Region()
rg2.load_from_fits(region2_fname)

if len(srcs_ids1)>0:
    rg1.tt_sources.add_index("rg_src_id")
    idx1_srcs = rg1.tt_sources.loc_indices["rg_src_id", srcs_ids1]
    tt_srcs1 = Table(rg1.tt_sources[idx1_srcs])
else:
    tt_srcs1 = rg.tt_sources
    
coords1 = SkyCoord(tt_srcs1["ra"], tt_srcs1["dec"], frame="icrs")
coords2 = SkyCoord(rg2.tt_sources["ra"], rg2.tt_sources["dec"], frame="icrs")
idx1, d2d, d3d = coords2.match_to_catalog_sky(coords1)

rg_src_id1 = tt_srcs1[idx1]["rg_src_id"]
rg2.tt_sources["rg_src_id1"]=rg_src_id1
rg2.tt_sources["dist_id1"]=d2d.to(uu.arcsec)
sel = rg2.tt_sources["dist_id1"] < dist_max
#display(rg2.tt_sources["rg_src_id","rg_src_id1","dist_id1"][sel])
rg2.tt_sources["rg_src_id","rg_src_id1","dist_id1"][sel].pprint_all()
print("srcs_ids = [", end="")
for src_id in rg2.tt_sources["rg_src_id"][sel].data:
    print(src_id,",", end="")
print("]")
print(sel.sum(), "counterparts found, for", len(srcs_ids1), "sources" )

rg_src_id rg_src_id1        dist_id1       
                             arcsec        
--------- ---------- ----------------------
     8215      13580 2.5469069747238155e-11
    32448      50388  2.289999370653732e-11
    54184      79477    0.01262109044767649
    98746     138783                    0.0
   151796     204419                    0.0
   166179     221349                    0.0
   172775     228958  4.092245627246673e-10
   193067     253214                    0.0
   199832     261213  9.999674717101199e-11
   259271     331466 1.0910436600057058e-10
   265150     340464  1.658875934128795e-10
   297278     381336 1.0196898108668681e-10
   305192     390864 1.8172551574936445e-10
   388172     489430  2.519746251174169e-10
   432606     540235  1.144999685326866e-11
   472623     583838 1.1103972181471528e-10
   535864     659787  1.144999685326866e-11
   541266     666224 2.8715562699842645e-11
   625693     764827                    0.0
  1521738    1805505 1.021671960